<h2> Data science Capstone Project </h2>
<h3>Opening a new coffee shop in Cincinnati</h3>

Build a dataframe of neighborhoods in Cincinnati, Ohio by web scraping the data from Wikipedia page<br>
Get the geographical coordinates of the neighborhoods<br>
Obtain the venue data for the neighborhoods from Foursquare API<br>
Explore and cluster the neighborhoods<br>
Select the best cluster to open a new coffee shop<br>

<b>Importing Libraries</b>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
! conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

<b>Scrapping data from wikipedia to a data frame</b>

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Cincinnati").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = []

In [6]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
cn_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cn_df.shape

(53, 1)

<b> Getting the geographical coordinates</b>

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ohio'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in cn_df["Neighborhood"].tolist() ]

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
cn_df['Latitude'] = df_coords['Latitude']
cn_df['Longitude'] = df_coords['Longitude']
cn_df.drop([0], axis = 0, inplace = True)

In [14]:
# check the neighborhoods and the coordinates
print(cn_df.shape)
cn_df.head()

(52, 3)


Neighborhood  Latitude  Longitude
1         Avondale, Cincinnati  39.14771  -84.49490
2        Bond Hill, Cincinnati  39.17460  -84.46715
3       California, Cincinnati  39.06536  -84.42365
4  Camp Washington, Cincinnati  39.13691  -84.53730
5         Carthage, Cincinnati  39.19733  -84.48062

In [15]:
# save the DataFrame as CSV file
cn_df.to_csv("cn_df.csv", index=False)

<b> Creating a Map of neignborhoods of Cincinnati </b>

In [17]:
# get the coordinates of Kuala Lumpur
address = 'Cincinnati, Ohio'

geolocator = Nominatim(user_agent="myapplication")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cincinnati, Ohio {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cincinnati, Ohio 39.1014537, -84.5124602.


In [18]:
# create map of Toronto using latitude and longitude values
map_cn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(cn_df['Latitude'], cn_df['Longitude'], cn_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_cn)  
    
map_cn

In [19]:
# save the map as HTML file
map_cn.save('map_cn.html')

<b> Use the Foursquare API to explore the neighborhoods </b> 

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = '2VQLJ5RKLN0E2WO5FVGVSMWUF1BFMBIY5MZDA0HJKTHHOBZT' # your Foursquare ID
CLIENT_SECRET = 'KZCKC4NL4GYQBEJZO3JMYFGDEK4XFY4YZQRWSSZZPKIRBDNZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2VQLJ5RKLN0E2WO5FVGVSMWUF1BFMBIY5MZDA0HJKTHHOBZT
CLIENT_SECRET:KZCKC4NL4GYQBEJZO3JMYFGDEK4XFY4YZQRWSSZZPKIRBDNZ


<b> Getting top 200 locations(venues) of the particular neighborhood within 2000 meters radius</b> 

In [21]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(cn_df['Latitude'], cn_df['Longitude'], cn_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        


In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(50)

(2979, 7)


Neighborhood  Latitude  Longitude  \
0   Avondale, Cincinnati  39.14771   -84.4949   
1   Avondale, Cincinnati  39.14771   -84.4949   
2   Avondale, Cincinnati  39.14771   -84.4949   
3   Avondale, Cincinnati  39.14771   -84.4949   
4   Avondale, Cincinnati  39.14771   -84.4949   
5   Avondale, Cincinnati  39.14771   -84.4949   
6   Avondale, Cincinnati  39.14771   -84.4949   
7   Avondale, Cincinnati  39.14771   -84.4949   
8   Avondale, Cincinnati  39.14771   -84.4949   
9   Avondale, Cincinnati  39.14771   -84.4949   
10  Avondale, Cincinnati  39.14771   -84.4949   
11  Avondale, Cincinnati  39.14771   -84.4949   
12  Avondale, Cincinnati  39.14771   -84.4949   
13  Avondale, Cincinnati  39.14771   -84.4949   
14  Avondale, Cincinnati  39.14771   -84.4949   
15  Avondale, Cincinnati  39.14771   -84.4949   
16  Avondale, Cincinnati  39.14771   -84.4949   
17  Avondale, Cincinnati  39.14771   -84.4949   
18  Avondale, Cincinnati  39.14771   -84.4949   
19  Avondale, Cincinnati  39.14771   -84.4949   
20  Avondale, Cincinnati  39.14771   -84.4949   
21  Avondale, Cincinnati  39.14771   -84.4949   
22  Avondale, Cincinnati  39.14771   -84.4949   
23  Avondale, Cincinnati  39.14771   -84.4949   
24  Avondale, Cincinnati  39.14771   -84.4949   
25  Avondale, Cincinnati  39.14771   -84.4949   
26  Avondale, Cincinnati  39.14771   -84.4949   
27  Avondale, Cincinnati  39.14771   -84.4949   
28  Avondale, Cincinnati  39.14771   -84.4949   
29  Avondale, Cincinnati  39.14771   -84.4949   
30  Avondale, Cincinnati  39.14771   -84.4949   
31  Avondale, Cincinnati  39.14771   -84.4949   
32  Avondale, Cincinnati  39.14771   -84.4949   
33  Avondale, Cincinnati  39.14771   -84.4949   
34  Avondale, Cincinnati  39.14771   -84.4949   
35  Avondale, Cincinnati  39.14771   -84.4949   
36  Avondale, Cincinnati  39.14771   -84.4949   
37  Avondale, Cincinnati  39.14771   -84.4949   
38  Avondale, Cincinnati  39.14771   -84.4949   
39  Avondale, Cincinnati  39.14771   -84.4949   
40  Avondale, Cincinnati  39.14771   -84.4949   
41  Avondale, Cincinnati  39.14771   -84.4949   
42  Avondale, Cincinnati  39.14771   -84.4949   
43  Avondale, Cincinnati  39.14771   -84.4949   
44  Avondale, Cincinnati  39.14771   -84.4949   
45  Avondale, Cincinnati  39.14771   -84.4949   
46  Avondale, Cincinnati  39.14771   -84.4949   
47  Avondale, Cincinnati  39.14771   -84.4949   
48  Avondale, Cincinnati  39.14771   -84.4949   
49  Avondale, Cincinnati  39.14771   -84.4949   

                                   VenueName  VenueLatitude  VenueLongitude  \
0                                 Hippo Cove      39.145257      -84.506104   
1                        Dobsa Giraffe Ridge      39.143495      -84.506975   
2                              Jungle Trails      39.146071      -84.506643   
3          Cincinnati Zoo & Botanical Garden      39.142740      -84.509266   
4     Marge Schott-Unnewehr Elephant Reserve      39.143109      -84.508114   
5                 Kroger Lords of the Arctic      39.145949      -84.507424   
6   Cincinnati Children's Hospital Cafeteria      39.140524      -84.501905   
7                                 Cat Canyon      39.145130      -84.509434   
8               Otto M Budig Manatee Springs      39.146575      -84.509065   
9                              Skyline Chili      39.143275      -84.508362   
10                        Wings of the World      39.145282      -84.506753   
11                                     Zoup!      39.135218      -84.503345   
12                   The Cincinnati Zoo Shop      39.143187      -84.508796   
13                             Reptile House      39.144165      -84.509250   
14                    Listermann Brewing Co.      39.146470      -84.472237   
15                       Mecklenburg Gardens      39.132676      -84.502615   
16                      P&G Discovery Forest      39.142756      -84.507252   
17                                Fries Cafe      39.139708      -84.512773   
18         

<b> Check how many venues were returned for a particular neighborhood </b> 

In [23]:
venues_df.groupby(["Neighborhood"]).count().head()

Latitude  Longitude  VenueName  VenueLatitude  \
Neighborhood                                                                 
Avondale, Cincinnati               77         77         77             77   
Bond Hill, Cincinnati              47         47         47             47   
CUF, Cincinnati                    92         92         92             92   
California, Cincinnati             34         34         34             34   
Camp Washington, Cincinnati        88         88         88             88   

                             VenueLongitude  VenueCategory  
Neighborhood                                                
Avondale, Cincinnati                     77             77  
Bond Hill, Cincinnati                    47             47  
CUF, Cincinnati                          92             92  
California, Cincinnati                   34             34  
Camp Washington, Cincinnati              88             88

In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 248 uniques categories.


In [25]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Zoo Exhibit', 'Zoo', 'Cafeteria', 'Hot Dog Joint', 'Soup Place',
       'Gift Shop', 'Brewery', 'German Restaurant', 'Exhibit', 'Bar',
       'American Restaurant', 'Hotel', 'Coffee Shop',
       'Mexican Restaurant', 'High School', 'Discount Store', 'Hotel Bar',
       'Fried Chicken Joint', 'Pizza Place', 'Sandwich Place',
       'Donut Shop', 'Supermarket', 'Burrito Place', 'Train Station',
       'ATM', 'Ice Cream Shop', 'College Cafeteria', 'Video Store',
       'Bank', 'Pharmacy', 'Fast Food Restaurant',
       'Eastern European Restaurant', 'Gas Station', 'Golf Course',
       'Bakery', 'Movie Theater', 'Athletics & Sports',
       'Convenience Store', 'Mediterranean Restaurant', 'Theater',
       'Basketball Court', 'Food Truck', 'Grocery Store',
       'Motorcycle Shop', 'Breakfast Spot', 'Food', 'Burger Joint',
       'Planetarium', 'Roller Rink', 'Furniture / Home Store', 'Diner',
       'Chinese Restaurant', 'Seafood Restaurant', 'Italian Restaurant',
       'Bus St

In [26]:
# check if the results contain "Shopping Mall"
"Coffee Shop" in venues_df['VenueCategory'].unique()

True

<b> One hot encoding of the venues by venue categories </b>

In [39]:
# one hot encoding
cn_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cn_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cn_onehot.columns[-1]] + list(cn_onehot.columns[:-1])
cn_onehot = cn_onehot[fixed_columns]

print(kl_onehot.shape)
cn_onehot.head()

(2979, 249)


Neighborhoods  ATM  Accessories Store  Airport  Airport Terminal  \
0  Avondale, Cincinnati    0                  0        0                 0   
1  Avondale, Cincinnati    0                  0        0                 0   
2  Avondale, Cincinnati    0                  0        0                 0   
3  Avondale, Cincinnati    0                  0        0                 0   
4  Avondale, Cincinnati    0                  0        0                 0   

   American Restaurant  Antique Shop  Aquarium  Arcade  \
0                    0             0         0       0   
1                    0             0         0       0   
2                    0             0         0       0   
3                    0             0         0       0   
4                    0             0         0       0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Automotive Shop  BBQ Joint  \
0                 0                   0                0          0   
1                 0                   0                0          0   
2                 0                   0                0          0   
3                 0                   0                0          0   
4                 0                   0                0          0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    0               0                 0   
2           0       0     0    0               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Court  Basketball Stadium  Bavarian Restaurant  Beach  Beer Bar  \
0                 0                   0                    0      0         0   
1                 0                   0                    0      0         0   
2                 0                   0                    0      0         0   
3                 0                   0                    0      0         0   
4                 0                   0                    0      0         0   

   Beer Garden  Big Box Store  Bistro  Boat or Ferry  Bookstore  \
0            0              0       0              0          0   
1            0              0       0              0          0   
2            0              0       0              0          0   
3            0              0       0              0          0   
4            0              0       0              0          0   

   Border Crossing  Boutique  Bowling Alley  Breakfast Spot  Brewery  \
0                0         0              0               0        0   
1                0         0              0               0        0   
2                0         0              0               0        0   
3                0         0              0               0        0   
4                0         0              0               0        0   

   Bridal Shop  Bubble Tea Shop  Buffet  Building  Burger Joint  \
0            0                0       0         0             0   
1            0                0       0         0             0   
2            0                0       0         0             0   
3            0                0       0         0             0   
4            0                0       0         0             0   

   Burrito Place  Bus Station  Business Service  Butcher  Cafeteria  Café  \
0              0            0                 0        0          0     0   
1            

<b> Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</b>

In [40]:
cn_grouped = cn_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(cn_grouped.shape)
cn_grouped.head()

(52, 249)


Neighborhoods       ATM  Accessories Store  Airport  \
0         Avondale, Cincinnati  0.025974            0.00000      0.0   
1        Bond Hill, Cincinnati  0.021277            0.00000      0.0   
2              CUF, Cincinnati  0.000000            0.01087      0.0   
3       California, Cincinnati  0.000000            0.00000      0.0   
4  Camp Washington, Cincinnati  0.011364            0.00000      0.0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Arcade  \
0               0.0             0.025974           0.0       0.0     0.0   
1               0.0             0.042553           0.0       0.0     0.0   
2               0.0             0.043478           0.0       0.0     0.0   
3               0.0             0.058824           0.0       0.0     0.0   
4               0.0             0.034091           0.0       0.0     0.0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                     0.0          0.0         0.0                  0.0   
1                     0.0          0.0         0.0                  0.0   
2                     0.0          0.0         0.0                  0.0   
3                     0.0          0.0         0.0                  0.0   
4                     0.0          0.0         0.0                  0.0   

   Asian Restaurant  Athletics & Sports  Automotive Shop  BBQ Joint  \
0               0.0            0.012987          0.00000        0.0   
1               0.0            0.000000          0.00000        0.0   
2               0.0            0.000000          0.01087        0.0   
3               0.0            0.029412          0.00000        0.0   
4               0.0            0.000000          0.00000        0.0   

   Bagel Shop    Bakery      Bank       Bar  Baseball Field  Baseball Stadium  \
0    0.000000  0.012987  0.012987  0.012987             0.0               0.0   
1    0.000000  0.000000  0.042553  0.000000             0.0               0.0   
2    0.000000  0.000000  0.000000  0.032609             0.0               0.0   
3    0.000000  0.000000  0.000000  0.029412             0.0               0.0   
4    0.022727  0.000000  0.022727  0.056818             0.0               0.0   

   Basketball Court  Basketball Stadium  Bavarian Restaurant  Beach  Beer Bar  \
0          0.012987                 0.0                  0.0    0.0       0.0   
1          0.000000                 0.0                  0.0    0.0       0.0   
2          0.000000                 0.0                  0.0    0.0       0.0   
3          0.000000                 0.0                  0.0    0.0       0.0   
4          0.000000                 0.0                  0.0    0.0       0.0   

   Beer Garden  Big Box Store   Bistro  Boat or Ferry  Bookstore  \
0          0.0            0.0  0.00000            0.0   0.000000   
1          0.0            0.0  0.00000            0.0   0.000000   
2          0.0            0.0  0.01087            0.0   0.000000   
3          0.0            0.0  0.00000            0.0   0.000000   
4          0.0            0.0  0.00000            0.0   0.011364   

   Border Crossing  Boutique  Bowling Alley  Breakfast Spot   Brewery  \
0         0.000000       0.0        0.00000        0.000000  0.012987   
1         0.000000       0.0        0.00000        0.021277  0.000000   
2         0.000000       0.0        0.01087        0.021739  0.000000   
3         0.029412       0.0        0.00000        0.000000  0.029412   
4         0.000000       0.0        0.00000        0.022727  0.000000   

   Bridal Shop  Bubble Tea Shop    Buffet  Building  Burger Joint  \
0     0.000000         0.000000  0.000000       0.0      0.000000   
1     0.000000         0.000000  0.000000       0.0      0.042553   
2     0.000000         0.000000  0.000000       0.0      0.043478   
3     0.029412         0.000000  0.029412       0.0      0.029412   
4     0.000000         0.011364  0.000000       0.0      0.000000   

   Burrito P

In [41]:
len(cn_grouped[cn_grouped["Coffee Shop"] > 0])

37

<b>Create a new DataFrame for Coffee shop data only</b>

In [42]:
cn_mall = cn_grouped[["Neighborhoods","Coffee Shop"]]
cn_mall.head()

Neighborhoods  Coffee Shop
0         Avondale, Cincinnati     0.038961
1        Bond Hill, Cincinnati     0.042553
2              CUF, Cincinnati     0.021739
3       California, Cincinnati     0.000000
4  Camp Washington, Cincinnati     0.022727

<b> Cluster Neighborhoods </b><br>
k means algorithm with k =3

In [43]:
# set number of clusters
kclusters = 3

cn_clustering = cn_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 0, 2, 0, 2, 2, 2, 2], dtype=int32)

In [44]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
cn_merged = cn_mall.copy()

# add clustering labels
cn_merged["Cluster Labels"] = kmeans.labels_

In [45]:
cn_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
cn_merged.head()

Neighborhood  Coffee Shop  Cluster Labels
0         Avondale, Cincinnati     0.038961               1
1        Bond Hill, Cincinnati     0.042553               1
2              CUF, Cincinnati     0.021739               2
3       California, Cincinnati     0.000000               0
4  Camp Washington, Cincinnati     0.022727               2

In [46]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cn_merged = cn_merged.join(cn_df.set_index("Neighborhood"), on="Neighborhood")

print(cn_merged.shape)
cn_merged.head() # check the last columns!

(52, 5)


Neighborhood  Coffee Shop  Cluster Labels  Latitude  \
0         Avondale, Cincinnati     0.038961               1  39.14771   
1        Bond Hill, Cincinnati     0.042553               1  39.17460   
2              CUF, Cincinnati     0.021739               2  39.16200   
3       California, Cincinnati     0.000000               0  39.06536   
4  Camp Washington, Cincinnati     0.022727               2  39.13691   

   Longitude  
0  -84.49490  
1  -84.46715  
2  -84.45689  
3  -84.42365  
4  -84.53730

In [47]:
# sort the results by Cluster Labels
print(cn_merged.shape)
cn_merged.sort_values(["Cluster Labels"], inplace=True)
cn_merged

(52, 5)


Neighborhood  Coffee Shop  Cluster Labels   Latitude  \
51           Winton Hills, Cincinnati     0.000000               0  39.189320   
15          English Woods, Cincinnati     0.000000               0  39.140220   
14          East Westwood, Cincinnati     0.000000               0  39.149910   
29        North Fairmount, Cincinnati     0.000000               0  39.135890   
39              Riverside, Cincinnati     0.000000               0  39.077460   
40               Roselawn, Cincinnati     0.000000               0  39.194780   
10               Covedale, Cincinnati     0.000000               0  39.121430   
22               Millvale, Cincinnati     0.000000               0  39.144430   
41            Sayler Park, Cincinnati     0.000000               0  39.102910   
44        South Fairmount, Cincinnati     0.000000               0  39.127560   
5                Carthage, Cincinnati     0.000000               0  39.197330   
46  Villages at Roll Hill, Cincinnati     0.000000               0  39.259697   
3              California, Cincinnati     0.000000               0  39.065360   
24             Mount Airy, Cincinnati     0.000000               0  39.191720   
49          Western Hills, Cincinnati     0.000000               0  39.123986   
32                     Over-the-Rhine     0.040000               1  39.111450   
31                 Oakley, Cincinnati     0.060000               1  39.153330   
30              Northside, Cincinnati     0.043478               1  39.165470   
33          Paddock Hills, Cincinnati     0.054545               1  39.164370   
37             Price Hill, Cincinnati     0.060606               1  39.110340   
35              Pill Hill, Cincinnati     0.045455               1  39.166175   
36         Pleasant Ridge, Cincinnati     0.042553               1  39.183580   
42            Sedamsville, Cincinnati     0.041667               1  39.093040   
43     South Cumminsville, Cincinnati     0.040000               1  39.153460   
48               West End, Cincinnati     0.040000               1  39.108820   
34              Pendleton, Cincinnati     0.040000               1  39.110302   
28         North Avondale, Cincinnati     0.037736               1  39.154320   
0                Avondale, Cincinnati     0.038961               1  39.147710   
26          Mount Lookout, Cincinnati     0.050000               1  39.128730   
1               Bond Hill, Cincinnati     0.042553               1  39.174600   
11                Downtown Cincinnati     0.040000               1  39.097230   
13      East Walnut Hills, Cincinnati     0.054795               1  39.126660   
16               Evanston, Cincinnati     0.041667               1  39.144520   
18              Hyde Park, Cincinnati     0.050000               1  39.142190   
25     Mount Auburn Historic District     0.040000               1  39.120340   
21           Madisonville, Cincinnati     0.050847               1  39.157380   
23            Mount Adams, Cincinnati     0.050000               1  39.109200   
19        Kennedy Heights, Cincinnati     0.086957               1  39.185740   
17               Hartwell, Cincinnati     0.025000               2  39.213390   
50               Westwood, Cincinnati     0.016129               2  39.148570   
2                     CUF, Cincinnati     0.021739               2  39.162000   
47           Walnut Hills, Cincinnati     0.022727               2  39.127200   
4         Camp Washington, Cincinnati     0.022727               2  39.136910   
45   Spring Grove Village, Cincinnati     0.025641               2  39.165078   
7            College Hill, Cincinnati     0.022222               2  39.202870   
8       Columbia-Tusculum, Cincinnati     0.028571               2  39.115690   
9              Corryville, Cincinnati     0.030928               2  39.134320   
27       Mount Washington, Cincinnati     0.032258               2  39.086250   
38             Queensgate, Cincinnati     0.020000      

<b> Lets visualize the clusters </b>

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cn_merged['Latitude'], cn_merged['Longitude'], cn_merged['Neighborhood'], cn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<b> Examining Clusters </b><br>
Cluster 0

In [51]:
cn_merged.loc[cn_merged['Cluster Labels'] == 0]

Neighborhood  Coffee Shop  Cluster Labels   Latitude  \
51           Winton Hills, Cincinnati          0.0               0  39.189320   
15          English Woods, Cincinnati          0.0               0  39.140220   
14          East Westwood, Cincinnati          0.0               0  39.149910   
29        North Fairmount, Cincinnati          0.0               0  39.135890   
39              Riverside, Cincinnati          0.0               0  39.077460   
40               Roselawn, Cincinnati          0.0               0  39.194780   
10               Covedale, Cincinnati          0.0               0  39.121430   
22               Millvale, Cincinnati          0.0               0  39.144430   
41            Sayler Park, Cincinnati          0.0               0  39.102910   
44        South Fairmount, Cincinnati          0.0               0  39.127560   
5                Carthage, Cincinnati          0.0               0  39.197330   
46  Villages at Roll Hill, Cincinnati          0.0               0  39.259697   
3              California, Cincinnati          0.0               0  39.065360   
24             Mount Airy, Cincinnati          0.0               0  39.191720   
49          Western Hills, Cincinnati          0.0               0  39.123986   

    Longitude  
51 -84.511070  
15 -84.557210  
14 -84.564190  
29 -84.556870  
39 -84.602230  
40 -84.462430  
10 -84.604080  
22 -84.552070  
41 -84.681310  
44 -84.555380  
5  -84.480620  
46 -84.345048  
3  -84.423650  
24 -84.568930  
49 -84.611668

<b> cluster 1 </b>

In [53]:
cn_merged.loc[cn_merged['Cluster Labels'] == 1]

Neighborhood  Coffee Shop  Cluster Labels   Latitude  \
32                  Over-the-Rhine     0.040000               1  39.111450   
31              Oakley, Cincinnati     0.060000               1  39.153330   
30           Northside, Cincinnati     0.043478               1  39.165470   
33       Paddock Hills, Cincinnati     0.054545               1  39.164370   
37          Price Hill, Cincinnati     0.060606               1  39.110340   
35           Pill Hill, Cincinnati     0.045455               1  39.166175   
36      Pleasant Ridge, Cincinnati     0.042553               1  39.183580   
42         Sedamsville, Cincinnati     0.041667               1  39.093040   
43  South Cumminsville, Cincinnati     0.040000               1  39.153460   
48            West End, Cincinnati     0.040000               1  39.108820   
34           Pendleton, Cincinnati     0.040000               1  39.110302   
28      North Avondale, Cincinnati     0.037736               1  39.154320   
0             Avondale, Cincinnati     0.038961               1  39.147710   
26       Mount Lookout, Cincinnati     0.050000               1  39.128730   
1            Bond Hill, Cincinnati     0.042553               1  39.174600   
11             Downtown Cincinnati     0.040000               1  39.097230   
13   East Walnut Hills, Cincinnati     0.054795               1  39.126660   
16            Evanston, Cincinnati     0.041667               1  39.144520   
18           Hyde Park, Cincinnati     0.050000               1  39.142190   
25  Mount Auburn Historic District     0.040000               1  39.120340   
21        Madisonville, Cincinnati     0.050847               1  39.157380   
23         Mount Adams, Cincinnati     0.050000               1  39.109200   
19     Kennedy Heights, Cincinnati     0.086957               1  39.185740   

    Longitude  
32 -84.515220  
31 -84.424860  
30 -84.541130  
33 -84.478390  
37 -84.576030  
35 -84.546280  
36 -84.424690  
42 -84.572920  
43 -84.550050  
48 -84.532720  
34 -84.506903  
28 -84.488440  
0  -84.494900  
26 -84.430220  
1  -84.467150  
11 -84.519640  
13 -84.473320  
16 -84.469470  
18 -84.434230  
25 -84.508280  
21 -84.391030  
23 -84.496390  
19 -84.408190

<b> Cluster 2 </b>

In [54]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

Neighborhood  Coffee Shop  Cluster Labels
2                    CUF, Cincinnati     0.021739               2
4        Camp Washington, Cincinnati     0.022727               2
6                Clifton, Cincinnati     0.014493               2
7           College Hill, Cincinnati     0.022222               2
8      Columbia-Tusculum, Cincinnati     0.028571               2
9             Corryville, Cincinnati     0.030928               2
12              East End, Cincinnati     0.022727               2
17              Hartwell, Cincinnati     0.025000               2
20               Linwood, Cincinnati     0.025641               2
27      Mount Washington, Cincinnati     0.032258               2
38            Queensgate, Cincinnati     0.020000               2
45  Spring Grove Village, Cincinnati     0.025641               2
47          Walnut Hills, Cincinnati     0.022727               2
50              Westwood, Cincinnati     0.016129               2

<b> Observations: </b>

Most of the Coffee Shops are concentrated in the central area of Cincinnati, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no Coffee Shops in the neighborhoods. This represents a great opportunity and high potential areas to open new Coffee Shops as there is very little to no competition from existing malls. Meanwhile, Coffee Shops in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new Coffee Shops in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Coffee Shops in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of Coffee Shops and suffering from intense competition.